## Install requirements

In [ ]:
!python.exe -m pip install --upgrade pip

In [ ]:
%pip install -r requirements-torch-windows.txt -U
%pip install -r requirements.txt -U

## Init stable diffusion

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

def initdiffusionpipeline(model_id):
    # Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")
    return pipe

#diffusionpipeline=initdiffusionpipeline("stabilityai/stable-diffusion-2-1")
diffusionpipeline=initdiffusionpipeline("dreamlike-art/dreamlike-diffusion-1.0")

In [ ]:
from diffusers import StableDiffusionUpscalePipeline

def initupscalepipeline(model_id):
    pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipeline.enable_attention_slicing() 
    pipeline = pipeline.to("cuda")
    return pipeline

upscalepipeline=initupscalepipeline("stabilityai/stable-diffusion-x4-upscaler")

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline

def initimg2imgpipeline(model_id):
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")
    return pipe

img2imgpipeline = initimg2imgpipeline("runwayml/stable-diffusion-v1-5")

In [ ]:
import re
def saveimage(image, prompt, seed, postfix=""):
    cleanprompt = re.sub("( |,)", '-', prompt)
    fname = f"images/{cleanprompt}-{seed}-{postfix}.png"
    image.save(fname)
    print(fname)
    display(image)
    #image.show() # open outside

In [ ]:
def generate(prompt,nbimages=1,seed=42):
    images = []
    for i in range(nbimages):
        generator = torch.Generator("cuda").manual_seed(seed+i)
        image = diffusionpipeline(prompt,num_images_per_prompt=1, generator=generator).images[0]
        saveimage(image, prompt, seed+i)
        images.append(image)
    #images = diffusionpipeline(prompt,num_images_per_prompt=nbimages, generator=generator).images
    return images

In [ ]:
def derivate(srcimg, prompt, nbimages=1,seed=42):
    images = []
    for i in range(nbimages):
        generator = torch.Generator("cuda").manual_seed(seed+i)
        image = img2imgpipeline(prompt=prompt, image=srcimg, strength=0.75, guidance_scale=7.5,num_images_per_prompt=1, generator=generator).images[0]
        saveimage(image, prompt, seed+i, "derived")
        images.append(image)
    #images = img2imgpipeline(prompt=prompt, image=srcimg, strength=0.75, guidance_scale=7.5,num_images_per_prompt=nbimages, generator=generator).images
    return images

In [ ]:
def upscale(prompt,srcimg,seed=42):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = upscalepipeline(prompt=prompt, image=srcimg,generator=generator).images[0]
    saveimage(image, prompt, seed, "upscaled")
    return image

## Run

In [ ]:
prompt="imagine a prehistoric octopus lurking in the deep green blue sea, deep blue shadows and sun rays, undersea jungle, high definition rendering"
images=generate(prompt,2)

## Derivate

In [ ]:
srcimg = images[1]
derivateimages = derivate(srcimg, prompt,2)